In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import cmudict
from textblob import TextBlob
from nltk.tokenize import sent_tokenize, word_tokenize


In [29]:
df = pd.read_excel('Input.xlsx')
df

,URL_ID,URL
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...
4,Netclan20241021,https://insights.blackcoffer.com/development-o...
...,...,...
142,Netclan20241159,https://insights.blackcoffer.com/population-an...
143,Netclan20241160,https://insights.blackcoffer.com/google-lsa-ap...
144,Netclan20241161,https://insights.blackcoffer.com/healthcare-da...
145,Netclan20241162,https://insights.blackcoffer.com/budget-sales-...


# Creating function for data extraction of article

# Extracting data in .txt format from each article

# Creating a title feature in Dataframe



In [ ]:

# Function to get article data and save it to a file
def get_article_data(url):
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        title = soup.find('h1', class_='entry-title')  
        text = soup.find('div', class_='td-pb-span8 td-main-content')
        
        # If title and text are found
        if title and text:
            title = title.text.strip()
            text = text.text.strip()
            filename = title.replace(" ", "_").replace("/", "_").replace(":", "") + '.txt'   
            
            # Ensure the directory exists
            os.makedirs('article data', exist_ok=True)
            
            # Define file path
            file_path = f'article data/{filename}'
            
            # Save the article to a text file
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(f"Title: {title}\n\n")
                f.write(text)
                print(f"Saved article '{title}' to {file_path}")
                
            return file_path  # Return the file path after saving
            
        return None  # Return None if title or text is not found
    except Exception as e:
        print(f"Failed to retrieve article from {url}. Error: {e}")
        return None

# Function to handle the download and saving of articles using ThreadPoolExecutor
def save_articles(df):
    with ThreadPoolExecutor() as executor:
        # Get the list of file paths by applying the get_article_data function to all URLs
        file_paths = list(executor.map(get_article_data, df['URL']))
        
        # Create a new column in the DataFrame for file paths
        df['title_file'] = file_paths

# Call the function
save_articles(df)

# Display the DataFrame to verify the file path column
print(df[['URL', 'title_file']])


Saved article 'Securing Sensitive Financial Data with Privacy-Preserving Machine Learning for Predictive Analytics' to article data/Securing_Sensitive_Financial_Data_with_Privacy-Preserving_Machine_Learning_for_Predictive_Analytics.txt
Saved article 'Development of EA Robot for Automated Trading' to article data/Development_of_EA_Robot_for_Automated_Trading.txt
Saved article 'ROAS Dashboard for Campaign-Wise Google Ads Budget Tracking Using Google Ads AP' to article data/ROAS_Dashboard_for_Campaign-Wise_Google_Ads_Budget_Tracking_Using_Google_Ads_AP.txt
Failed to retrieve article from https://insights.blackcoffer.com/efficient-processing-and-analysis-of-financial-data-from-pdf-files-addressing-formatting-inconsistencies-and-ensuring-data-integrity-for-a-toyota-dealership-management-firm/. Error: [Errno 2] No such file or directory: 'article data/Efficient_Processing_and_Analysis_of_Financial_Data_from_PDF_Files_Addressing_Formatting_Inconsistencies_and_Ensuring_Data_Integrity_for_a_Toy

In [33]:
df['title_file'].head()

0    article data/AI_and_ML-Based_YouTube_Analytics...
1    article data/Enhancing_Front-End_Features_and_...
2    article data/ROAS_Dashboard_for_Campaign-Wise_...
3                                                 None
4    article data/Development_of_EA_Robot_for_Autom...
Name: title_file, dtype: object

In [34]:
# Define the folder path
folder_path = 'MasterDictionary/'

# Read the negative words file
with open(f'{folder_path}/negative-words.txt', 'r') as file:
    negative_words = file.read().splitlines()  # Read each line and create a list

# Read the positive words file
with open(f'{folder_path}/positive-words.txt', 'r') as file:
    positive_words = file.read().splitlines()  # Read each line and create a list

# Check the results
print("Negative Words:", negative_words[:5])  # Print first 5 negative words
print("Positive Words:", positive_words[:5])  # Print first 5 positive words


Negative Words: ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable']
Positive Words: ['a+', 'abound', 'abounds', 'abundance', 'abundant']


# Making few more features for the feature extraction of the words that is positive words, negative words, complex words

In [35]:
def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""  # Return an empty string if there's an error

# Function to check and extract positive/negative words
def find_words(text, word_list):
    words_in_text = text.split()  # Split the text into individual words
    return [word for word in words_in_text if word.lower() in word_list]

# Read the content of each file in the 'file_path' column
df['text'] = df['title_file'].apply(read_text_file)

# Apply the function to create new columns in the DataFrame for positive and negative words
df['positive_words'] = df['text'].apply(lambda x: find_words(x, positive_words))
df['negative_words'] = df['text'].apply(lambda x: find_words(x, negative_words))

# Optionally, count the number of positive/negative words
df['positive_word_count'] = df['positive_words'].apply(len)
df['negative_word_count'] = df['negative_words'].apply(len)

# Display the dataframe with the new columns


Error reading None: expected str, bytes or os.PathLike object, not NoneType
Error reading None: expected str, bytes or os.PathLike object, not NoneType


In [36]:
df

,URL_ID,URL,title_file,text,positive_words,negative_words,positive_word_count,negative_word_count
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,article data/AI_and_ML-Based_YouTube_Analytics...,Title: AI and ML-Based YouTube Analytics and C...,"[leading, integrated, Improved]","[Cloud, Cloud]",3,2
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,article data/Enhancing_Front-End_Features_and_...,Title: Enhancing Front-End Features and Functi...,"[Improved, leading, lead, patient, user-friend...","[static, issues, strictly, issues, strictly, C...",15,6
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,article data/ROAS_Dashboard_for_Campaign-Wise_...,Title: ROAS Dashboard for Campaign-Wise Google...,"[leading, efficiently, streamlined, leading, c...","[lacks, difficulties, cloud, slack, failure, C...",10,8
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,None,,[],[],0,0
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,article data/Development_of_EA_Robot_for_Autom...,Title: Development of EA Robot for Automated T...,"[free, free, seamless, free, appropriate, like...","[limited, risk]",13,2
...,...,...,...,...,...,...,...,...
142,Netclan20241159,https://insights.blackcoffer.com/population-an...,article data/Population_and_Community_Survey_o...,Title: Population and Community Survey of Amer...,"[leading, proper, strong, Sublime, Advanced, E...","[unable, Cloud, plot, plot, plot, plot]",12,6
143,Netclan20241160,https://insights.blackcoffer.com/google-lsa-ap...,article data/Google_LSA_API_Data_Automation_an...,Title: Google LSA API Data Automation and Dash...,"[leading, lead, lead, lead, refresh, like, Sub...","[Regression, Cloud, Regression, missed, Cloud,...",19,18
144,Netclan20241161,https://insights.blackcoffer.com/healthcare-da...,article data/Healthcare_Data_Analysis.txt,Title: Healthcare Data Analysis\n\nClient Back...,"[leading, survivor, better, patient, patient, ...","[died, death, death, manipulation, Death, died...",10,9
145,Netclan20241162,https://insights.blackcoffer.com/budget-sales-...,"article data/Budget,_Sales_KPI_Dashboard_using...","Title: Budget, Sales KPI Dashboard using Power...",[],[],0,0


In [ ]:

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\gomes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:


# Download stopwords if you haven't already
nltk.download('stopwords')
nltk.download('punkt')

# Get the list of English stopwords
stop_words = set(stopwords.words('english'))

# Function to clean text by removing stop words
def remove_stopwords(text):
    # Tokenize the text
    words = word_tokenize(text.lower())  # Convert to lowercase and tokenize
    # Remove stop words and non-alphabetic characters (optional)
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]
    return filtered_words

# Apply the function to the 'text' column and create a new column 'filtered_words'
df['filtered_words'] = df['text'].apply(remove_stopwords)

# Display the updated dataframe
print(df[['text', 'filtered_words']])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gomes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gomes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                                  text  \
0    Title: AI and ML-Based YouTube Analytics and C...   
1    Title: Enhancing Front-End Features and Functi...   
2    Title: ROAS Dashboard for Campaign-Wise Google...   
3                                                        
4    Title: Development of EA Robot for Automated T...   
..                                                 ...   
142  Title: Population and Community Survey of Amer...   
143  Title: Google LSA API Data Automation and Dash...   
144  Title: Healthcare Data Analysis\n\nClient Back...   
145  Title: Budget, Sales KPI Dashboard using Power...   
146  Title: Amazon Buy Bot, an Automation AI tool t...   

                                        filtered_words  
0    [title, ai, youtube, analytics, content, creat...  
1    [title, enhancing, features, functionality, im...  
2    [title, roas, dashboard, google, ads, budget, ...  
3                                                   []  
4    [title, devel

In [14]:
df

,URL_ID,URL,title_file,text,positive_words,negative_words,positive_word_count,negative_word_count,filtered_words
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,article data/AI_and_ML-Based_YouTube_Analytics...,Title: AI and ML-Based YouTube Analytics and C...,"[leading, integrated, Improved]","[Cloud, Cloud]",3,2,"[title, ai, youtube, analytics, content, creat..."
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,article data/Enhancing_Front-End_Features_and_...,Title: Enhancing Front-End Features and Functi...,"[Improved, leading, lead, patient, user-friend...","[static, issues, strictly, issues, strictly, C...",15,6,"[title, enhancing, features, functionality, im..."
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,article data/ROAS_Dashboard_for_Campaign-Wise_...,Title: ROAS Dashboard for Campaign-Wise Google...,"[leading, efficiently, streamlined, leading, c...","[lacks, difficulties, cloud, slack, failure, C...",10,8,"[title, roas, dashboard, google, ads, budget, ..."
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,None,,[],[],0,0,[]
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,article data/Development_of_EA_Robot_for_Autom...,Title: Development of EA Robot for Automated T...,"[free, free, seamless, free, appropriate, like...","[limited, risk]",13,2,"[title, development, ea, robot, automated, tra..."
...,...,...,...,...,...,...,...,...,...
142,Netclan20241159,https://insights.blackcoffer.com/population-an...,article data/Population_and_Community_Survey_o...,Title: Population and Community Survey of Amer...,"[leading, proper, strong, Sublime, Advanced, E...","[unable, Cloud, plot, plot, plot, plot]",12,6,"[title, population, community, survey, america..."
143,Netclan20241160,https://insights.blackcoffer.com/google-lsa-ap...,article data/Google_LSA_API_Data_Automation_an...,Title: Google LSA API Data Automation and Dash...,"[leading, lead, lead, lead, refresh, like, Sub...","[Regression, Cloud, Regression, missed, Cloud,...",19,18,"[title, google, lsa, api, data, automation, da..."
144,Netclan20241161,https://insights.blackcoffer.com/healthcare-da...,article data/Healthcare_Data_Analysis.txt,Title: Healthcare Data Analysis\n\nClient Back...,"[leading, survivor, better, patient, patient, ...","[died, death, death, manipulation, Death, died...",10,9,"[title, healthcare, data, analysis, client, ba..."
145,Netclan20241162,https://insights.blackcoffer.com/budget-sales-...,"article data/Budget,_Sales_KPI_Dashboard_using...","Title: Budget, Sales KPI Dashboard using Power...",[],[],0,0,"[title, budget, sales, kpi, dashboard, using, ..."


In [15]:
df['filtered_words']

0      [title, ai, youtube, analytics, content, creat...
1      [title, enhancing, features, functionality, im...
2      [title, roas, dashboard, google, ads, budget, ...
3                                                     []
4      [title, development, ea, robot, automated, tra...
                             ...                        
142    [title, population, community, survey, america...
143    [title, google, lsa, api, data, automation, da...
144    [title, healthcare, data, analysis, client, ba...
145    [title, budget, sales, kpi, dashboard, using, ...
146    [title, amazon, buy, bot, automation, ai, tool...
Name: filtered_words, Length: 147, dtype: object

# feaure of complex words

In [16]:
import syllapy 
def count_complex_words_from_list(filtered_words):
    complex_words = []
    
    # Iterate through the list of words
    for word in filtered_words:
        syllable_count = syllapy.count(word)  
        if syllable_count >= 3: 
            complex_words.append(word)

    return complex_words

df['complex_words'] = df['filtered_words'].apply(count_complex_words_from_list)


df['complex_word_count'] = df['complex_words'].apply(len)

# Display the dataframe with the new feature
print(df[['filtered_words', 'complex_words', 'complex_word_count']].head())

                                      filtered_words  \
0  [title, ai, youtube, analytics, content, creat...   
1  [title, enhancing, features, functionality, im...   
2  [title, roas, dashboard, google, ads, budget, ...   
3                                                 []   
4  [title, development, ea, robot, automated, tra...   

                                       complex_words  complex_word_count  
0  [analytics, creation, optimizing, subscriber, ...                  86  
1  [enhancing, features, functionality, experienc...                 150  
2  [backgroundclient, usaindustry, itproducts, se...                 106  
3                                                 []                   0  
4  [development, automated, objective, advisor, a...                 170  


In [17]:
def calculate_word_count(text):
    # Split the text into words and count the number of words
    return len(text.split())

# Apply the function to your dataframe
df['word_count'] = df['text'].apply(calculate_word_count)


In [ ]:

# Load the CMU Pronouncing Dictionary to get syllable counts for words
nltk.download('cmudict')
pron_dict = cmudict.dict()
# Function to calculate FOG Index using existing columns
def calculate_fog_index(row):
    # Get the sentence count from the 'text' column
    sentences = re.split(r'[.!?]', row['text'])
    sentence_count = len([s for s in sentences if s.strip() != ''])  # Exclude empty sentences

    # Get word_count and complex_word_count from the existing columns
    word_count = row['word_count']
    complex_word_count = row['complex_word_count']

    # Calculate FOG index
    if sentence_count == 0 or word_count == 0:  # Avoid division by zero
        return 0
    fog_index = 0.4 * ((word_count / sentence_count) + 100 * (complex_word_count / word_count))
    return fog_index



def calculate_avg_words_per_sentence(row):
    sentences = row['text']  # Assuming 'text' is the original text column
    sentence_list = nltk.sent_tokenize(sentences)  # Tokenize text into sentences
    word_count = sum(len(nltk.word_tokenize(sentence)) for sentence in sentence_list)  # Total words in all sentences
    sentence_count = len(sentence_list)  # Number of sentences

    # To avoid division by zero if there are no sentences
    if sentence_count == 0:
        return 0
    avg_words_per_sentence = word_count / sentence_count
    return int(avg_words_per_sentence)  


def syllable_count(word):
    word = word.lower()
    if word in pron_dict:
        # Return the number of syllables in the word
        return max([len(list(y for y in x if y[-1].isdigit())) for x in pron_dict[word]])
    else:
        # If the word is not found in the dictionary, we return an estimate based on the length of the word
        return len([char for char in word if char in "aeiou"])

# Function to calculate the percentage of complex words
def calculate_complex_percentage(row):
    complex_word_count = 0
    words = row['filtered_words']  # Using the filtered_text (which is a list of words)
    
    for word in words:
        if syllable_count(word) > 2:  # Consider words with more than 2 syllables as complex
            complex_word_count += 1
    
    word_count = row['word_count']
    
    # Calculate Percentage of Complex Words
    if word_count == 0:  # Avoid division by zero
        return 0
    percentage_complex_words = (complex_word_count / word_count) * 100
    return percentage_complex_words

# Apply the function to your dataframe to create the "Avg Words Per Sentence" column
df['avg_words_per_sentence'] = df.apply(calculate_avg_words_per_sentence, axis=1)
df['fog_index'] = df.apply(calculate_fog_index, axis=1)
df['percentage_complex_words'] = df.apply(calculate_complex_percentage, axis=1)


[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\gomes\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


# Subjectivity score

In [ ]:


# Function to calculate subjectivity score
def calculate_subjectivity_score(text):
    blob = TextBlob(text)  # Create a TextBlob object
    return blob.sentiment.subjectivity  # Get the subjectivity score

# Apply the function to calculate subjectivity score for each row in the 'text' column
df['subjectivity_score'] = df['text'].apply(calculate_subjectivity_score)



In [20]:
personal_pronouns = ['I', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them', 'my', 'your', 'his', 'her', 'its', 'our', 'their']

# Function to extract the list of personal pronouns from the text
def extract_personal_pronouns(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Filter and collect personal pronouns found in the text
    pronouns_in_text = [word for word in words if word.lower() in personal_pronouns]
    
    return pronouns_in_text

# Apply the function to create a new column 'personal_pronouns_list' in the DataFrame
df['personal_pronouns_list'] = df['text'].apply(extract_personal_pronouns)

# Display the dataframe with the new feature
print(df[['text', 'personal_pronouns_list']].head())

                                                text  \
0  Title: AI and ML-Based YouTube Analytics and C...   
1  Title: Enhancing Front-End Features and Functi...   
2  Title: ROAS Dashboard for Campaign-Wise Google...   
3                                                      
4  Title: Development of EA Robot for Automated T...   

                              personal_pronouns_list  
0  [IT, IT, IT, their, them, Our, them, their, IT...  
1  [our, our, us, our, our, our, me, you, you, yo...  
2                       [IT, IT, IT, it, it, IT, my]  
3                                                 []  
4                                       [it, It, my]  


In [21]:
df.head()

,URL_ID,URL,title_file,text,positive_words,negative_words,positive_word_count,negative_word_count,filtered_words,complex_words,complex_word_count,word_count,avg_words_per_sentence,fog_index,percentage_complex_words,subjectivity_score,personal_pronouns_list
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,article data/AI_and_ML-Based_YouTube_Analytics...,Title: AI and ML-Based YouTube Analytics and C...,"[leading, integrated, Improved]","[Cloud, Cloud]",3,2,"[title, ai, youtube, analytics, content, creat...","[analytics, creation, optimizing, subscriber, ...",86,275,65,18.620202,33.818182,0.283902,"[IT, IT, IT, their, them, Our, them, their, IT..."
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,article data/Enhancing_Front-End_Features_and_...,Title: Enhancing Front-End Features and Functi...,"[Improved, leading, lead, patient, user-friend...","[static, issues, strictly, issues, strictly, C...",15,6,"[title, enhancing, features, functionality, im...","[enhancing, features, functionality, experienc...",150,901,71,14.850177,16.759156,0.431083,"[our, our, us, our, our, our, me, you, you, yo..."
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,article data/ROAS_Dashboard_for_Campaign-Wise_...,Title: ROAS Dashboard for Campaign-Wise Google...,"[leading, efficiently, streamlined, leading, c...","[lacks, difficulties, cloud, slack, failure, C...",10,8,"[title, roas, dashboard, google, ads, budget, ...","[backgroundclient, usaindustry, itproducts, se...",106,461,32,15.556018,26.464208,0.264918,"[IT, IT, IT, it, it, IT, my]"
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,None,,[],[],0,0,[],[],0,0,0,0.000000,0.000000,0.000000,[]
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,article data/Development_of_EA_Robot_for_Autom...,Title: Development of EA Robot for Automated T...,"[free, free, seamless, free, appropriate, like...","[limited, risk]",13,2,"[title, development, ea, robot, automated, tra...","[development, automated, objective, advisor, a...",170,723,24,14.146239,24.066390,0.316736,"[it, It, my]"


# Syllabel per word

In [22]:
def count_syllables(word):
    return syllapy.count(word)

# Function to calculate average syllables per word in a text
def syllables_per_word(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Calculate total syllables in all words
    total_syllables = sum(count_syllables(word) for word in words)
    
    # Calculate average syllables per word (to avoid division by zero)
    avg_syllables = total_syllables / len(words) if words else 0
    
    return int(avg_syllables)

# Apply the function to create a new column 'syllables_per_word' in the DataFrame
df['syllables_per_word'] = df['text'].apply(syllables_per_word)

df.head()

,URL_ID,URL,title_file,text,positive_words,negative_words,positive_word_count,negative_word_count,filtered_words,complex_words,complex_word_count,word_count,avg_words_per_sentence,fog_index,percentage_complex_words,subjectivity_score,personal_pronouns_list,syllables_per_word
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,article data/AI_and_ML-Based_YouTube_Analytics...,Title: AI and ML-Based YouTube Analytics and C...,"[leading, integrated, Improved]","[Cloud, Cloud]",3,2,"[title, ai, youtube, analytics, content, creat...","[analytics, creation, optimizing, subscriber, ...",86,275,65,18.620202,33.818182,0.283902,"[IT, IT, IT, their, them, Our, them, their, IT...",1
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,article data/Enhancing_Front-End_Features_and_...,Title: Enhancing Front-End Features and Functi...,"[Improved, leading, lead, patient, user-friend...","[static, issues, strictly, issues, strictly, C...",15,6,"[title, enhancing, features, functionality, im...","[enhancing, features, functionality, experienc...",150,901,71,14.850177,16.759156,0.431083,"[our, our, us, our, our, our, me, you, you, yo...",1
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,article data/ROAS_Dashboard_for_Campaign-Wise_...,Title: ROAS Dashboard for Campaign-Wise Google...,"[leading, efficiently, streamlined, leading, c...","[lacks, difficulties, cloud, slack, failure, C...",10,8,"[title, roas, dashboard, google, ads, budget, ...","[backgroundclient, usaindustry, itproducts, se...",106,461,32,15.556018,26.464208,0.264918,"[IT, IT, IT, it, it, IT, my]",1
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,None,,[],[],0,0,[],[],0,0,0,0.000000,0.000000,0.000000,[],0
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,article data/Development_of_EA_Robot_for_Autom...,Title: Development of EA Robot for Automated T...,"[free, free, seamless, free, appropriate, like...","[limited, risk]",13,2,"[title, development, ea, robot, automated, tra...","[development, automated, objective, advisor, a...",170,723,24,14.146239,24.066390,0.316736,"[it, It, my]",1


# Polarity score, Average sentence length

In [ ]:
def calculate_polarity_score(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # Polarity score between -1 and 1

# Apply the function to create a new column 'polarity_score' in the DataFrame
def calculate_avg_sentence_length(text):
    sentences = sent_tokenize(text)  # Tokenize text into sentences
    words = word_tokenize(text)      # Tokenize text into words
    word_count = len([word for word in words if word.isalnum()])  # Count only words, excluding punctuation
    sentence_count = len(sentences)  # Count number of sentences
    avg_sentence_length = word_count / sentence_count if sentence_count > 0 else 0  # Calculate average length
    return avg_sentence_length

# Apply the function to create a new column 'avg_sentence_length' in the DataFrame
df['avg_sentence_length'] = df['text'].apply(calculate_avg_sentence_length)

df['polarity_score'] = df['text'].apply(calculate_polarity_score)
df.head()

,URL_ID,URL,title_file,text,positive_words,negative_words,positive_word_count,negative_word_count,filtered_words,complex_words,complex_word_count,word_count,avg_words_per_sentence,fog_index,percentage_complex_words,subjectivity_score,personal_pronouns_list,syllables_per_word,avg_sentence_length,polarity_score
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,article data/AI_and_ML-Based_YouTube_Analytics...,Title: AI and ML-Based YouTube Analytics and C...,"[leading, integrated, Improved]","[Cloud, Cloud]",3,2,"[title, ai, youtube, analytics, content, creat...","[analytics, creation, optimizing, subscriber, ...",86,275,65,18.620202,33.818182,0.283902,"[IT, IT, IT, their, them, Our, them, their, IT...",1,52.200000,0.027462
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,article data/Enhancing_Front-End_Features_and_...,Title: Enhancing Front-End Features and Functi...,"[Improved, leading, lead, patient, user-friend...","[static, issues, strictly, issues, strictly, C...",15,6,"[title, enhancing, features, functionality, im...","[enhancing, features, functionality, experienc...",150,901,71,14.850177,16.759156,0.431083,"[our, our, us, our, our, our, me, you, you, yo...",1,61.214286,0.166357
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,article data/ROAS_Dashboard_for_Campaign-Wise_...,Title: ROAS Dashboard for Campaign-Wise Google...,"[leading, efficiently, streamlined, leading, c...","[lacks, difficulties, cloud, slack, failure, C...",10,8,"[title, roas, dashboard, google, ads, budget, ...","[backgroundclient, usaindustry, itproducts, se...",106,461,32,15.556018,26.464208,0.264918,"[IT, IT, IT, it, it, IT, my]",1,27.687500,0.065501
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,None,,[],[],0,0,[],[],0,0,0,0.000000,0.000000,0.000000,[],0,0.000000,0.000000
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,article data/Development_of_EA_Robot_for_Autom...,Title: Development of EA Robot for Automated T...,"[free, free, seamless, free, appropriate, like...","[limited, risk]",13,2,"[title, development, ea, robot, automated, tra...","[development, automated, objective, advisor, a...",170,723,24,14.146239,24.066390,0.316736,"[it, It, my]",1,19.657143,0.118347


In [24]:
df = df.drop(columns = ['title_file','text'])

In [25]:
df.to_csv('final_blackcoffer.csv')

In [26]:
df

,URL_ID,URL,positive_words,negative_words,positive_word_count,negative_word_count,filtered_words,complex_words,complex_word_count,word_count,avg_words_per_sentence,fog_index,percentage_complex_words,subjectivity_score,personal_pronouns_list,syllables_per_word,avg_sentence_length,polarity_score
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,"[leading, integrated, Improved]","[Cloud, Cloud]",3,2,"[title, ai, youtube, analytics, content, creat...","[analytics, creation, optimizing, subscriber, ...",86,275,65,18.620202,33.818182,0.283902,"[IT, IT, IT, their, them, Our, them, their, IT...",1,52.200000,0.027462
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,"[Improved, leading, lead, patient, user-friend...","[static, issues, strictly, issues, strictly, C...",15,6,"[title, enhancing, features, functionality, im...","[enhancing, features, functionality, experienc...",150,901,71,14.850177,16.759156,0.431083,"[our, our, us, our, our, our, me, you, you, yo...",1,61.214286,0.166357
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,"[leading, efficiently, streamlined, leading, c...","[lacks, difficulties, cloud, slack, failure, C...",10,8,"[title, roas, dashboard, google, ads, budget, ...","[backgroundclient, usaindustry, itproducts, se...",106,461,32,15.556018,26.464208,0.264918,"[IT, IT, IT, it, it, IT, my]",1,27.687500,0.065501
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,[],[],0,0,[],[],0,0,0,0.000000,0.000000,0.000000,[],0,0.000000,0.000000
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,"[free, free, seamless, free, appropriate, like...","[limited, risk]",13,2,"[title, development, ea, robot, automated, tra...","[development, automated, objective, advisor, a...",170,723,24,14.146239,24.066390,0.316736,"[it, It, my]",1,19.657143,0.118347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Netclan20241159,https://insights.blackcoffer.com/population-an...,"[leading, proper, strong, Sublime, Advanced, E...","[unable, Cloud, plot, plot, plot, plot]",12,6,"[title, population, community, survey, america...","[population, community, america, backgroundcli...",207,905,80,18.199171,22.651934,0.399814,"[them, their, them, we, our, my, It]",1,71.750000,0.062403
143,Netclan20241160,https://insights.blackcoffer.com/google-lsa-ap...,"[leading, lead, lead, lead, refresh, like, Sub...","[Regression, Cloud, Regression, missed, Cloud,...",19,18,"[title, google, lsa, api, data, automation, da...","[automation, dashboarding, backgroundclient, m...",278,1372,46,16.175545,21.720117,0.251640,"[its, their, them, their, it, we, it, our, my,...",1,40.375000,0.043007
144,Netclan20241161,https://insights.blackcoffer.com/healthcare-da...,"[leading, survivor, better, patient, patient, ...","[died, death, death, manipulation, Death, died...",10,9,"[title, healthcare, data, analysis, client, ba...","[analysis, backgroundclient, usaindustry, cons...",66,415,56,13.578837,15.903614,0.338480,"[we, us, we, we, us, we, we, we, us, our, we, ...",1,50.250000,0.026348
145,Netclan20241162,https://insights.blackcoffer.com/budget-sales-...,[],[],0,0,"[title, budget, sales, kpi, dashboard, using, ...","[descriptionweekly, clustered, actual, actual,...",44,163,186,75.997546,26.380368,0.414035,[],1,143.000000,-0.008772
